In [10]:
!pip install pandas langchain faiss-cpu sentence-transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 44.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 78.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 86.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 61.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 86.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstallin

In [13]:
!pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 40.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.0 MB/s eta 0:00:00


In [21]:
!pip install emoji

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 11.8 MB/s eta 0:00:00


In [22]:
import pandas as pd
import re
import emoji

# Define the CSV file path
csv_filepath = "/content/volunteer-descriptions.csv"

# Load the CSV
df = pd.read_csv(csv_filepath)

# Clean column names (remove spaces)
df.columns = df.columns.str.strip()

# Remove duplicates
df = df.drop_duplicates()

# Drop rows with missing values in 'Volunteer_ID' or 'Cleaned_Description'
df = df.dropna(subset=["Volunteer_ID", "Description"])

# Function to clean text
def clean_text(text):
    # Convert text to lowercase
    text = text.lower()

    # Remove emojis
    text = emoji.replace_emoji(text, replace='')  # Remove emojis

    # Remove punctuation and special characters (except spaces and alphanumeric)
    text = re.sub(r"[^a-zA-Z0-9\s]", "", text)

    # Remove extra spaces
    text = " ".join(text.split())

    return text

# Apply the clean_text function to the 'Cleaned_Description' column
df["Description"] = df["Description"].apply(clean_text)

# Save the cleaned CSV
cleaned_filepath = "/content/newdata.csv"
df.to_csv(cleaned_filepath, index=False)

print(f"✅ CSV cleaned and saved as: {cleaned_filepath}")
print(df.head())  # Display the first few rows


✅ CSV cleaned and saved as: /content/newdata.csv
   Volunteer_ID                                        Description
0             1  hi there ive been teaching english as a second...
1             2  im pretty good with graphic design stuff photo...
2             3  so im a software developer python js some c an...
3             4  passionate about saving the environment organi...
4             5  first aid instructor certified looking to volu...


In [23]:
import pandas as pd

# Define the correct file path
csv_filepath = "/content/newdata.csv"

# Read and preview the data
df = pd.read_csv(csv_filepath)

# Show the first few rows
print(df.head())

# Ensure required columns exist
if "Volunteer_ID" not in df.columns or "Description" not in df.columns:
    raise ValueError("CSV must contain ' Volunteer_ID' and 'Description' columns")

   Volunteer_ID                                        Description
0             1  hi there ive been teaching english as a second...
1             2  im pretty good with graphic design stuff photo...
2             3  so im a software developer python js some c an...
3             4  passionate about saving the environment organi...
4             5  first aid instructor certified looking to volu...


In [26]:
from langchain.embeddings import HuggingFaceEmbeddings

class VolunteerEmbeddings:
    def __init__(self, model_name="sentence-transformers/all-MiniLM-L6-v2"):
        self.csv_file = csv_filepath  # Use the fixed file path
        self.model = HuggingFaceEmbeddings(model_name=model_name)
        self.volunteer_data = self.load_data()
        self.embeddings = self.create_embeddings()

    def load_data(self):
        """Load volunteer descriptions from CSV."""
        df = pd.read_csv(self.csv_file)
        return df

    def create_embeddings(self):
        """Convert text descriptions to vector embeddings."""
        return self.volunteer_data["Description"].apply(lambda x: self.model.embed_query(str(x))).tolist()

# Initialize with the correct file path
embedder = VolunteerEmbeddings()
print("✅ Volunteer data processed successfully!")

✅ Volunteer data processed successfully!


In [28]:
import faiss
import numpy as np

class VolunteerMatcher:
    def __init__(self):
        """Initialize FAISS and load volunteer data."""
        self.embedding_model = VolunteerEmbeddings()
        self.volunteer_data = self.embedding_model.volunteer_data
        self.embeddings = np.array(self.embedding_model.embeddings)

        if len(self.embeddings) == 0:
            print("No embeddings available. Check data loading.")
            self.index = None
        else:
            self.volunteer_ids = self.volunteer_data["Volunteer_ID"].tolist()
            self.index = self.create_faiss_index()

    def create_faiss_index(self):
        """Create a FAISS index for similarity search."""
        dimension = self.embeddings.shape[1]  # Embedding vector size
        index = faiss.IndexFlatL2(dimension)
        index.add(self.embeddings)
        return index

    def find_similar_volunteers(self, query, top_k=3):
        """Find top-k similar volunteers."""
        if self.index is None:
            return {"error": "No data loaded"}

        query_embedding = np.array([self.embedding_model.model.embed_query(query)])
        distances, indices = self.index.search(query_embedding, top_k)

        results = []
        for idx, dist in zip(indices[0], distances[0]):
            volunteer_id = self.volunteer_ids[idx]
            description = self.volunteer_data.iloc[idx]["Description"]
            similarity_score = round(1 / (1 + dist), 2)  # Convert L2 distance to similarity
            results.append({"Volunteer_ID": volunteer_id, "Description": description, "similarity_score": similarity_score})

        return results

# Initialize matcher with correct file path
matcher = VolunteerMatcher()
print("✅ FAISS index created successfully!")


✅ FAISS index created successfully!


In [30]:
query = "Looking for volunteers skilled in graphic design to help with branding."
matches = matcher.find_similar_volunteers(query)

print("\nTop 3 Matches:")
for match in matches:
    print(f"ID: {match['Volunteer_ID']}, Description: {match['Description']} (Score: {match['similarity_score']})")



Top 3 Matches:
ID: 2, Description: im pretty good with graphic design stuff photoshop illustrator etc would love to help nonprofits with their promotional materials i work from home so can be flexible with my hours (Score: 0.5899999737739563)
ID: 7, Description: i do social media marketing for a living happy to help small charities with their fb insta tiktok etc can do like 5 hrs a week remotely probably dm me for my portfolio (Score: 0.47999998927116394)
ID: 63, Description: ux designer for tech company can help improve website experiences for nonprofit services remote work in evenings and some weekends (Score: 0.47999998927116394)


SAY YOU WANTED TO ACTUALLY FINETUNE THIS using LORA


In [ ]:
import os
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import get_peft_model, LoraConfig, PeftModel
from torch.utils.data import DataLoader
from transformers import Trainer, TrainingArguments
import torch
from dotenv import load_dotenv
from huggingface_hub import login
from transformers import AutoModel


# Load environment variables from .env file
load_dotenv()

# Get the Hugging Face token from the .env file
hf_token = os.getenv("HF_TOKEN")

# Login to Hugging Face using the token
if hf_token:
    login(token=hf_token)
else:
    print("Hugging Face token is missing.")

def fine_tune_lora_model():
    # Load dataset (cleaned data here)
    df = pd.read_csv('/content/newdata.csv')
    train_data = Dataset.from_pandas(df)

    # Load pre-trained model and tokenizer from Hugging Face
    tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-3B-Instruct")
    model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-3B-Instruct")

    # Prepare the dataset for fine-tuning
    def tokenize_function(examples):
        return tokenizer(examples['Description'], padding=True, truncation=True, max_length=512)

    tokenized_datasets = train_data.map(tokenize_function, batched=True)
    tokenized_datasets = tokenized_datasets.shuffle(seed=42)

    # Define LoRA configuration
    lora_config = LoraConfig(
        r=8,               # Rank
        lora_alpha=16,     # Scaling factor
        lora_dropout=0.1,  # Dropout rate
        bias="none",       # Bias to adjust in LoRA
    )

    # Apply LoRA to the model
    model = get_peft_model(model, lora_config)

    # Training arguments
    training_args = TrainingArguments(
        output_dir="./fine_tuned_model",
        evaluation_strategy="epoch",
        learning_rate=5e-5,
        per_device_train_batch_size=2,# reduced to speed up training
        num_train_epochs=1, #0nly 1 epoch for quick fine-tuning
        weight_decay=0.01, # Weight decay for regularization
        logging_dir='./logs', # Directory for logging
        logging_steps=10, # Logging frequency
        max_steps=100, # Maximum number of training steps
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_datasets,
    )

    # Fine-tune the model
    trainer.train()

    # Save the model after training
    model.save_pretrained('./fine_tuned_model')
    tokenizer.save_pretrained('./fine_tuned_model')
    print("Fine-tuning completed and model saved!")

if __name__ == '__main__':
    fine_tune_lora_model()
